In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [ ]:
def residual_block(filters,size):
    initializer=tf.random_normal_initializer(0,0.02)
    block=tf.keras.Sequential()
    block.add(tf.keras.layers.ZeroPadding2D(1))
    block.add(tf.keras.layers.Conv2D(filters,size,strides=1,padding='valid',kernel_initializer=initializer,use_bias=False))
    block.add(tf.keras.layers.BatchNormalization())
    block.add(tf.keras.layers.LeakyReLU())
    block.add(tf.keras.layers.ZeroPadding2D(1))
    block.add(tf.keras.layers.Conv2D(filters,size,strides=1,padding='valid',kernel_initializer=initializer,use_bias=False))
    block.add(tf.keras.layers.BatchNormalization())
    
    return block

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
          tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                                 kernel_initializer=initializer, use_bias=False))

    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    result.add(tf.keras.layers.LeakyReLU())

    return result

In [ ]:
def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
        tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                        padding='same',
                                        kernel_initializer=initializer,
                                        use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result

In [ ]:
def Generator():
    initializer=tf.random_normal_initializer(0,0.02)
    inputs=tf.keras.layers.Input(shape=[256,256,3])#标签
    
    residual_blocks=[
        residual_block(512,3),
        residual_block(512,3),
        residual_block(512,3),
        residual_block(512,3),
        residual_block(512,3),
        residual_block(512,3),
        residual_block(512,3),
        residual_block(512,3),
        residual_block(512,3)
    ]
    
    x1=inputs
    x1=tf.keras.layers.ZeroPadding2D(3)(x1)
    x1=tf.keras.layers.Conv2D(64,7,strides=1,padding='valid',kernel_initializer=initializer,use_bias=False)(x1)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1=tf.keras.layers.LeakyReLU()(x1)
    
    #下采样 此时输入为（256，256，64）
    x1=tf.keras.layers.Conv2D(128,3,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(x1)#(128,128,128)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1_1=tf.keras.layers.LeakyReLU()(x1)
    x1=tf.keras.layers.Conv2D(256,3,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(x1_1)#(64,64,256)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1_2=tf.keras.layers.LeakyReLU()(x1)
    x1=tf.keras.layers.Conv2D(512,3,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(x1_2)#(32,32,512)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1=tf.keras.layers.LeakyReLU()(x1)
    x1_3=x1
    
    #残差
    for block in residual_blocks:
        x11=block(x1)
        x1=tf.add(x1,x11)
    #上采样此时（32，32，256）
    x1=tf.keras.layers.Concatenate()([x1_3, x1])
    x1=tf.keras.layers.Conv2DTranspose(256,3,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(x1)#(64,64,256)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1=tf.keras.layers.LeakyReLU()(x1)
    
    x1=tf.keras.layers.Concatenate()([x1_2, x1])
    x1=tf.keras.layers.Conv2DTranspose(128,3,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(x1)#(128,128,128)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1=tf.keras.layers.LeakyReLU()(x1)
    
    x1=tf.keras.layers.Concatenate()([x1_1, x1])
    x1=tf.keras.layers.Conv2DTranspose(64,3,strides=2,padding='same',kernel_initializer=initializer,use_bias=False)(x1)#(256,256,64)
    x1=tf.keras.layers.BatchNormalization()(x1)
    x1=tf.keras.layers.LeakyReLU()(x1)
    
    x1=tf.keras.layers.Conv2D(3,7,strides=1,padding='same',kernel_initializer=initializer,use_bias=False,activation='tanh')(x1)
    
    
    
    
    return tf.keras.Model(inputs=inputs,outputs=x1)

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
    tar = tf.keras.layers.Input(shape=[256, 256, 3], name='target_image')

    x = tf.keras.layers.concatenate([inp, tar]) # (bs, 256, 256, channels*2)

    down1 = downsample(64, 4, False)(x) # (bs, 128, 128, 64)
    down2 = downsample(128, 4)(down1) # (bs, 64, 64, 128)
    down3 = downsample(256, 4)(down2) # (bs, 32, 32, 256)

    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
    conv = tf.keras.layers.Conv2D(512, 4, strides=1,
                                  kernel_initializer=initializer,
                                  use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

    batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

    leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

    last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                  kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

    return tf.keras.Model(inputs=[inp, tar], outputs=last)
